In [6]:
from utils import db_info

import numpy as np
import pandas as pd
import psycopg2
from sqlalchemy import *

#### 데이터베이스 접속
- data.ini를 사용해 hide variable 진행

In [7]:
connection_format = f'postgresql://{db_info["USER"]}:{db_info["PASSWORD"]}@{db_info["HOST"]}:{db_info["PORT"]}/{db_info["DB"]}'
        
client = create_engine(connection_format)

### Cluster_1
- 문제
    - 모든 환자에 대해 총 내원일수를 구하고 총 내원일수의 최대값과 총 내원일수 최대값을 가지는 환자수 출력
- 조건
    - visit_occurrence 테이블은 병원에 방문한 환자들의 병원 방문과 관련된 정보를 포함
    - `내원일수 = 방문종료일자 - 방문시작일자 + 1`로 계산

- 풀이
    - 분석함수를 사용해 총 내원일수들의 최대값을 구함 : max_visit_period
    - 내원한 환자 중 내원일수(visit_period)와 최대 내원일수(total_period)가 같은 환자들의 count 추출
    
#### RESULT
- 내원한 환자들의 최대 내원일수는 3731일 이며, 3731일간 내원한 환자 수는 1명이다.

In [3]:
QUERY_1 = """
WITH visit_period AS (
-- visit_period : 내원일수 컬럼을 추가한 테이블
SELECT person_id, (visit_end_date - visit_start_date + 1) AS visit_term
FROM visit_occurrence
)
, total_period AS (
-- total_period : visit_period 테이블에서 person_id별 총 내원일수를 구하고 내림차순으로 정렬 뒤 최대 총 내원일수를 구한 테이블
SELECT SUM(visit_term) AS sum_term
FROM visit_period
GROUP BY person_id
ORDER BY sum_term DESC
LIMIT 1
)

SELECT COUNT(DISTINCT vp.person_id) AS cnt, AVG(tp.sum_term) AS max_sum_term
FROM visit_period vp, total_period tp
GROUP BY vp.person_id 
HAVING SUM(vp.visit_term) = (SELECT sum_term FROM total_period);
"""

In [4]:
cluster_1 = pd.read_sql(QUERY_1, con=client)
cluster_1

,max_visit_period,patient_count
0,3731,1


### Cluster_2
- 문제
    - 환자들이 진단 받은 상병 내역 중 첫글자는 (a, b, c, d, e)로 시작하고 중간에 'heart'가 포함된 상병 이름 출력
- 조건
    - 문자 검색시 대소문자 구분 X
    - 상병 이름을 중복없이 나열
- 풀이
    - 상병코드가 담긴 condition_occurrence 테이블과 상병이름이 담긴 concept 테이블간 조인
        - JOIN 인라인뷰에서 필요한 행과 열만 가져와 효율적인 쿼리
    - 상병 이름(concept_name) 중 첫자리가 a, b, c, d, e(`^[a-e]`) 이고, 중간에 heart(`%heart%`)가 들어간 행만 추출
        - 쿼리 길이와 cost를 고려해 like 함수로 할 수 있는 부분은 ilike 함수 사용, a~e의 첫문자열에 대한 조건은 정규표현식 이용
    - 중복 제거를 위해 DISTINCT 예약어 사용
        
#### RESULT
- 조건을 모두 만족하는 상병은 `Chronic congestive heart failure` 만 존재한다.

In [5]:
QUERY_2 = """
SELECT DISTINCT B.concept_name
FROM de.condition_occurrence A
JOIN (
    SELECT concept_id, concept_name
    FROM concept
    WHERE concept_name ~* '^[a-e]' AND concept_name ILIKE '%heart%'
) B
ON A.condition_concept_id = B.concept_id;
"""

In [6]:
cluster_2 = pd.read_sql(QUERY_2, con=client)
cluster_2

,concept_name
0,Chronic congestive heart failure


### Cluster_3
- 문제
    - 환자번호 '1891866' 환자의 처방된 약 종류별로 처음 시작일, 마지막 종료일, 복용일을 구하고 복용일이 긴 순으로 정렬
- 조건
    - drug_exposure 테이블은 환자가 병원에서 처방받은 약의 종류와 처방시작일, 종료일에 대한 정보 포함
    - 복용일 : (마지막 종료일과 처음시작일의 차이) + 1
- 풀이
    - 해당 환자의 약 종류별 처음 시작일 중 최소값 추출 : 해당 약을 최초로 복용한 날짜 : start_date
    - 해당 환자의 약 종류별 마지막 종료일 중 최대값 추출 : 해당 약을 최후로 복용할 날짜 : end_date
    - 인라인뷰를 사용해 복용일(term) 컬럼을 생성한 후 term 기준 내림차순 진행
    - 과정
        
#### RESULT
- 해당 환자는 19009384, 19030765, 40213154, 1539463, 40213227 에 대한 약을 복용하였다.
- 각 약에 대한 복용일은 14190, 3640, 3221, 2921, 1일 이다.

In [7]:
QUERY_3 = """
SELECT drug_concept_id, start_date, end_date, (end_date - start_date + 1) AS term
FROM (
    SELECT
          drug_concept_id
        , MIN(drug_exposure_start_date) AS start_date
        , MAX(drug_exposure_end_date) AS end_date
    FROM drug_exposure
    WHERE person_id = 1891866
    GROUP BY drug_concept_id
     ) AS A
ORDER BY term DESC;
"""

In [8]:
cluster_3 = pd.read_sql(QUERY_3, con=client)
cluster_3

,drug_concept_id,start_date,end_date,term
0,19009384,1959-12-01,1998-10-06,14190
1,19030765,1988-10-18,1998-10-05,3640
2,40213154,1989-09-12,1998-07-07,3221
3,1539463,1990-03-13,1998-03-11,2921
4,40213227,1993-01-05,1993-01-05,1


### Cluster_4
- 문제
    - 15가지의 약의 처방 건수와 각 약별로 짝지어진 약의 처방 건수 중 두번째 약의 처방 건수가 첫번째 약의 처방 건수보다 더 많은 첫번째 약품명을 출력
- 조건
    - drug_exposure 테이블은 환자가 병원에서 처방받은 약의 종류와 처방시작일, 종료일에 대한 정보 포함
    - drug_pair 테이블은 약별로 가장 많이 처방되는 약을 짝지어 놓은 테이블
        - drug_concept_id1(첫번째약 번호), drug_concept_id2(두번째약 번호)
    - 특정 15가지의 약에 대해서만 진행
    - ORDER BY : 두번쨰 약의 처방 건수 순으로 내림차순 진행    
- 풀이
    - WITH 임시 테이블
        - drug_list : drug_exposure에서 15가지 약에 대한 id, name별 count가 담긴 테이블
        - drugs : drug_list에서 id와 name 정보만 가져온 테이블
            - drug_concept_id(첫번째약 번호), concept_name(약품명)
        - prescription_count : drug_list에서 id와 count 정보만 가져온 테이블
            - drug_concept_id(첫번째약 번호), cnt(처방건수)
            
    <br/>
        
    - FROM절 인라인뷰(d_pc) : drugs와 prescription_count을 JOIN하여 첫번째 약들에 대한 처방 건수 출력(id1_cnt)
    - JOIN절 인라인뷰(dp_pc) : drug_pair와 prescription_count을 JOIN하여 짝지어진 두번째 약들에 대한 처방 건수 출력(id2_cnt)
    - 두 테이블의 첫번째 약에 대한 번호를 기준으로 JOIN한 뒤, id1_cnt보다 id2_cnt가 큰 약들만 추출
    - filtering된 약들 중 id2_cnt에 대해 내림차순한 약 이름(concept_name) 출력

#### RESULT
- 첫번째 약 처방 건수보다 짝지어진 두번째 약 처방 건수가 많은 약들은 총 10개가 존재한다.

In [9]:
QUERY_4 = """
WITH drug_list AS (
SELECT DISTINCT drug_concept_id, concept_name, count(*) AS cnt
FROM drug_exposure
JOIN concept
ON drug_concept_id = concept_id
WHERE concept_id IN (40213154, 19078106, 19009384, 40224172, 19127663, 1511248, 40169216, 1539463
                    , 19126352, 1539411, 1332419, 40163924, 19030765, 19106768, 19075601)
GROUP BY drug_concept_id, concept_name
ORDER BY count(*) DESC 
)
, drugs AS (
SELECT drug_concept_id, concept_name
FROM drug_list
)
, prescription_count AS (
SELECT drug_concept_id, cnt
FROM drug_list
)

SELECT concept_name
FROM (
    SELECT d.drug_concept_id, d.concept_name, pc.cnt AS id1_cnt
    FROM drugs d
    JOIN prescription_count pc
    ON d.drug_concept_id = pc.drug_concept_id
) AS d_pc
JOIN (
    SELECT drug_pair.*, pc.cnt AS id2_cnt
    FROM drug_pair
    JOIN prescription_count pc
    ON drug_pair.drug_concept_id2 = pc.drug_concept_id
) AS dp_pc
ON d_pc.drug_concept_id = dp_pc.drug_concept_id1
WHERE id1_cnt < id2_cnt
ORDER BY id2_cnt DESC;
"""

In [10]:
cluster_4 = pd.read_sql(QUERY_4, con=client)
cluster_4

,concept_name
0,hydrochlorothiazide 25 MG Oral Tablet
1,amlodipine 5 MG / hydrochlorothiazide 12.5 MG ...
2,hydrochlorothiazide 12.5 MG Oral Tablet
3,24 HR metformin hydrochloride 500 MG Extended ...
4,atenolol 50 MG / chlorthalidone 25 MG Oral Tab...
5,1 ML epoetin alfa 4000 UNT/ML Injection [Epogen]
6,120 ACTUAT fluticasone propionate 0.044 MG/ACT...
7,simvastatin 20 MG Oral Tablet
8,amlodipine 5 MG Oral Tablet
9,clopidogrel 75 MG Oral Tablet


### Cluster_5
- 문제
    - 조건에 해당하는 환자수 추출
- 조건
    - a. 제 2형 당뇨병을 진단받은 환자 중에
    - b. 18세 이상의 환자 중에
    - c. 진단을 받은 이후 Metformin을 90일 이상 복용한 환자수
        - drug_concept_id : 40163924
- 풀이
    - WITH 임시 테이블
        - diabetes_list : 제 2형 당뇨병을 진단받은 환자와 그 환자의 birth_year 정보를 추출한 테이블
            - TO_CHAR 함수를 통해 condition_start_date에 대한 year 데이터만 추출한 컬럼 생성 : start_condition_year
        - age_over_18 : diabetes_list 데이터에서 나이가 18세 이상인 환자만 추출한 테이블
            - 환자의 year_of_birth와 start_condition_year의 차이가 18 이상인 행만 추출
        - metformin : metformin 약을 처방 받은 환자와 복용일에 대한 테이블
            - 복용일(exposure_term) : drug_exposure_end_date - drug_exposure_start_date
            
    <br/>
    
    - age_over_18 테이블과 metformin 테이블을 person_id, visit_occurrence_id로 JOIN
        - 환자별로 여러번 방문하기때문에 person_id와 visit_id가 동시에 같은 행끼리 조인해야한다
    - 진단 이후 Metformin을 복용한 경우에 대해서만 계산
        - age_over_18.condition_start_date 보다 metformin.drug_exposure_start_date 이 크거나 같아야 한다
    - 필터링 된 행에 한하여 person_id별 Metformin의 복용일의 합계가 90일 이상인 그룹만 추출, 혹시 모를 중복을 위해 person_id에 대해 DISTINCT 진행
    - 인라인뷰를 통해 추출된 그룹의 person_id 기준 count를 계산
    
    <br/>
    
- 주의사항
    - 위의 쿼리를 만족하는 4명의 person_id에 대해 처방내역을 확인해보니 중복된 visit_occurrence_id가 존재하는 경우가 많음
    <img width="895" alt="스크린샷 2021-11-29 오후 3 04 59" src="https://user-images.githubusercontent.com/80459520/143817010-179690c4-d7ac-459d-93f1-4f0d92bbf0f2.png">
        
        - 복용일에 + 1을 진행하지 않음 : +1을 하고 진행하게 된다면 중복된 id를 가진 날짜에 대해 +2가 되어버림
        
        <br/>
        
        - 해결방안
            - 방법 1. 복용일에 +1을 하지 않고 진행 (해당 방법 선택) : 복용일 1일 차이로 조건에 부합되지 않은 환자가 없었기에
            - 방법 2. CASE문을 사용해 drug_exposure_start_date와 drug_exposure_end_date가 같은 행은 1로 계산하고, 다른 행은 drug_exposure_end_date - drug_exposure_start_date + 1로 계산후 groupby를 통해 person_id, visit_occurrence_id별 복용일의 합계 컬럼을 생성

#### RESULT
- 제 2형 당뇨병을 진단받고, Metformin을 90일 이상 복용한 18세 이상을 환자들의 수는 4명 이다.

In [6]:
QUERY_5 = """
WITH diabetes_list AS (
SELECT person_id, visit_occurrence_id, condition_start_date
     , TO_CHAR(condition_start_date, 'YYYY') AS start_condition_year
FROM condition_occurrence
WHERE condition_concept_id IN (3191208, 36684827, 3194332, 3193274, 43531010, 4130162
                              ,45766052, 45757474, 4099651, 4129519, 4063043, 4230254
                              ,4193704, 4304377, 201826, 3194082, 3192767)
)
, age_over_18 AS (
SELECT dl.person_id, dl.visit_occurrence_id, dl.condition_start_date
FROM diabetes_list dl
JOIN person p
ON dl.person_id = p.person_id
WHERE (dl.start_condition_year::INT - p.year_of_birth) >= 18
)
, metformin AS (
-- metformin : metformin 약을 처방 받은 환자와 복용일에 대한 테이블
SELECT person_id, visit_occurrence_id, drug_exposure_start_date
     , (drug_exposure_end_date - drug_exposure_start_date) AS exposure_term
FROM drug_exposure
WHERE drug_concept_id = 40163924
)

SELECT COUNT(*) AS person_cnt
FROM (
    SELECT DISTINCT ao.person_id
    FROM age_over_18 ao
    JOIN metformin m
    ON ao.person_id = m.person_id AND ao.visit_occurrence_id = m.visit_occurrence_id
    WHERE ao.condition_start_date <= m.drug_exposure_start_date
    GROUP BY ao.person_id
    HAVING SUM(m.exposure_term) >= 90
) AS A;
"""

# """
# -- 위의 쿼리를 만족하는 환자에 대해 처방내역을 확인해보는 쿼리
# SELECT person_id, drug_concept_id, drug_exposure_start_date, drug_exposure_end_date, visit_occurrence_id
#      , (drug_exposure_end_date - drug_exposure_start_date + 1) AS exposure_term
# FROM drug_exposure
# WHERE person_id IN (843873, 1317600, 1578321, 1826955) AND drug_concept_id = 40163924
# ORDER BY person_id, drug_exposure_start_date;
# """

In [7]:
cluster_5 = pd.read_sql(QUERY_5, con=client)
cluster_5

,person_cnt
0,4
